In [1]:
import sys
print(sys.executable)

/workspaces/Kaggle-Solution-Finder-Agent/app/.venv/bin/python


In [2]:
import os
from pathlib import Path

# 1) Redirect logs to eval_logs/
os.environ["LOGS_DIRECTORY"] = "eval_logs"

# 2) Make app/ importable
REPO_ROOT = Path.cwd().parent
sys.path.insert(0, str(REPO_ROOT / "app"))

from ingest import (read_repo_data,extract_completed_competitions,build_vector_index)
from search_agent import create_search_agent
from logs import log_interaction_to_file, LOG_DIR
from pydantic_ai import Agent
from logs import EvaluationChecklist

/workspaces/Kaggle-Solution-Finder-Agent/app/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
LOG_DIR.absolute()

PosixPath('/workspaces/Kaggle-Solution-Finder-Agent/eval/logs')

In [4]:
project_docs = read_repo_data(repo_owner="Khangtran94",repo_name="kaggle-solutions",branch="gh-pages")
completed_competitions = extract_completed_competitions(project_docs)
embedding_model, vindex = build_vector_index(completed_competitions)

Batches: 100%|██████████| 18/18 [01:06<00:00,  3.68s/it]


In [7]:
question_generation_prompt = """
You are an evaluation agent responsible for assessing the quality, correctness,
    and consistency of an AI assistant’s response to a user query.

    INPUTS YOU WILL RECEIVE:
    1. The original user query
    2. The system prompt used by the assistant
    3. The assistant’s final answer

    YOUR TASK:
    Evaluate whether the assistant’s answer satisfies the system prompt and the user query.

    EVALUATION CRITERIA (mandatory):

    1. QUERY SATISFACTION
    - Does the answer address ALL explicit constraints in the query?
    - Are numeric conditions (e.g., “more than 5 solutions”) respected?
    - Are no required elements missing?

    2. COMPLETENESS
    - Are all relevant items included?
    - Were any potential matches silently omitted?
    - If uncertainty exists, is it explicitly stated?

    3. CONSISTENCY & DETERMINISM
    - Would the same query reasonably yield the same set of results?
    - Is the ordering stable and logically justified?

    4. DOMAIN RELEVANCE
    - Are results correctly classified (Strong / Partial / Weak match)?
    - Are irrelevant items incorrectly included or relevant ones excluded?

    5. SOLUTION LISTING ACCURACY
    - Is the reported number of solutions consistent with the listed solutions?
    - Are links plausible and correctly associated with the competition?
    - Are all available solutions listed when required by the system prompt?

    6. TRANSPARENCY
    - Are assumptions, inferences, or uncertainties clearly disclosed?
    - Is there any hallucinated or unverifiable information?

    OUTPUT FORMAT (STRICT):
    Provide:
    - A verdict: PASS or FAIL
    - A brief justification for each evaluation criterion
    - A final summary highlighting the most critical issues (if any)

    DO NOT:
    - Rewrite the assistant’s answer
    - Introduce new competitions or facts
    - Assume missing information is correct

    Your role is to judge, not to fix.
""".strip()

from pydantic import BaseModel
from pydantic_ai import Agent

class QuestionsList(BaseModel):
    questions: list[str]

question_generate_agent = Agent(
    name="question_generate_agent",
    instructions=question_generation_prompt,
    model="gpt-4o-mini",
    output_type=QuestionsList)

import random
import json
# Sample 10 random completed competitions from the list
sample = random.sample(completed_competitions, 10)

# # Create the prompt docs
# prompt_docs = [
#     (
#         f"Title: {d.get('title', 'N/A')}\n"
#         f"Metric: {d.get('metric', 'N/A')}\n"
#         f"Link: {d.get('link', 'N/A')}\n"
#         f"Number of Solutions: {len(d['solutions']) if isinstance(d.get('solutions'), list) else d.get('solutions', 'N/A')}\n"
#     )
#     for d in sample
# ]

# # Join the formatted prompt docs into a single string
# prompt = "\n\n".join(prompt_docs)
# prompt

In [8]:
questions = []

for competition in sample:
    # Build prompt once per competition
    prompt = (
        f"Title: {competition.get('title', 'N/A')}\n"
        f"Metric: {competition.get('metric', 'N/A')}\n"
        f"Link: {competition.get('link', 'N/A')}\n"
        f"Number of Solutions: "
        f"{len(competition['solutions']) if isinstance(competition.get('solutions'), list) else competition.get('solutions', 'N/A')}\n"
    )

    # Generate questions
    result = await question_generate_agent.run(prompt)

    # Collect questions
    questions.extend(result.output.questions)

In [ ]:
# # Generate questions for each competition individually
# questions = []

# # Loop through each competition in the sample
# for competition in sample:
#     # Create the individual prompt for the competition
#     prompt = (
#         f"Title: {competition.get('title', 'N/A')}\n"
#         f"Metric: {competition.get('metric', 'N/A')}\n"
#         f"Link: {competition.get('link', 'N/A')}\n"
#         f"Number of Solutions: {len(competition['solutions']) if isinstance(competition.get('solutions'), list) else competition.get('solutions', 'N/A')}\n"
#     )
    
#     # Generate the questions for this competition
#     question_generated = await question_generate_agent.run(prompt)
    
#     # Append the questions to the final list
#     questions.extend(question_generated.output.questions)


# # Print the questions to check them
# print(questions)

40
['What does the Abstraction and Reasoning Challenge evaluation metric measure in the ARC Prize 2024 competition and what techniques could participants use to optimize it?', "How does the title 'ARC Prize 2024' reflect the underlying tasks and goals of this competition?", 'What are the key domains and topics covered in the ARC Prize 2024 competition, and how do they relate to artificial intelligence and reasoning?', 'With only 7 solutions submitted in the ARC Prize 2024 competition, what competitive strategies might impact the performance and diversity of approaches among participants?', 'What does the Weighted Multiclass Loss metric measure, and how can participants optimize their models to achieve a lower value for this metric in the Mayo Clinic - STRIP AI competition?', "How does the title 'Mayo Clinic - STRIP AI' reflect the focus of the project, and what implications does it have for the types of solutions being submitted?", 'What medical domain challenges are being addressed in

In [9]:
questions

["Does the assistant's answer satisfy the system prompt regarding the 'Carvana Image Masking Challenge' by providing the correct title, metric, link, and number of solutions?",
 "Does the assistant's answer explicitly address all constraints in the user query?",
 "Are all relevant items included in the assistant's response?",
 'Is the ordering of items stable and logically justified?',
 'Are results correctly classified as Strong / Partial / Weak match with no irrelevant items included?',
 'Is the reported number of solutions consistent with the listed solutions?',
 'Are all available solutions listed as required by the system prompt?',
 'Are assumptions, inferences, or uncertainties clearly disclosed?',
 "Is there any unverifiable information present in the assistant's response?",
 'PASS',
 "The assistant correctly identified one solution as required by the user query in connection to the 'NFL 1st and Future - Analytics.'",
 "Does the assistant's answer address ALL explicit constraint

In [11]:
query_agent = create_search_agent(embedding_model, vindex) 
query_agent 
for q in questions: 
    print(q) 
    answer_question = await query_agent.run(user_prompt = q) 
    log_interaction_to_file(query_agent, answer_question.new_messages())

Does the assistant's answer satisfy the system prompt regarding the 'Carvana Image Masking Challenge' by providing the correct title, metric, link, and number of solutions?
Does the assistant's answer explicitly address all constraints in the user query?
Are all relevant items included in the assistant's response?
Is the ordering of items stable and logically justified?
Are results correctly classified as Strong / Partial / Weak match with no irrelevant items included?
Is the reported number of solutions consistent with the listed solutions?
Are all available solutions listed as required by the system prompt?
Are assumptions, inferences, or uncertainties clearly disclosed?
Is there any unverifiable information present in the assistant's response?
PASS
The assistant correctly identified one solution as required by the user query in connection to the 'NFL 1st and Future - Analytics.'
Does the assistant's answer address ALL explicit constraints in the query?
Are numeric conditions (e.g., 